In [1]:
!gdown --id 1voLia8xeIwIQCX5XFGMU4Py7Tsy0AWMp #modified wikinews_extractor 
!unzip -qq wikinews_extractor.zip

!gdown --id  1XZUtCYus8umDXHZQgZkx_An5FOiVyRYr #modified run_parse_wikinews_i18n.sh

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1voLia8xeIwIQCX5XFGMU4Py7Tsy0AWMp
To: /content/wikinews_extractor.zip
100% 22.5k/22.5k [00:00<00:00, 25.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1XZUtCYus8umDXHZQgZkx_An5FOiVyRYr
To: /content/run_parse_wikinews_i18n.sh
100% 1.87k/1.87k [00:00<00:00, 2.32MB/s]


In [2]:
#download and preprocessing raw mewsli-9 dataset
%%bash
#download github repo for preprocess mewsli-9 dataset

gdown --id 1_W_B0kOG3bLnnXRluUa1sptUWRKcqsDy # mel.zip 
unzip -qq mel.zip

mv mel/mewsli-9 mewsli-9
mv mel/tools tools

sh mel/get-mewsli-9.sh
pip install -r wikinews_extractor/requirements.txt
mv -u run_parse_wikinews_i18n.sh mewsli-9/
bash mewsli-9/run_parse_wikinews_i18n.sh

Archive:  mewsli-9.zip
   creating: ./mewsli-9/output/dataset/ar/
  inflating: ./mewsli-9/output/dataset/ar/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/ar/docs.tsv  
   creating: ./mewsli-9/output/dataset/de/
  inflating: ./mewsli-9/output/dataset/de/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/de/docs.tsv  
   creating: ./mewsli-9/output/dataset/en/
  inflating: ./mewsli-9/output/dataset/en/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/en/docs.tsv  
   creating: ./mewsli-9/output/dataset/es/
  inflating: ./mewsli-9/output/dataset/es/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/es/docs.tsv  
   creating: ./mewsli-9/output/dataset/fa/
  inflating: ./mewsli-9/output/dataset/fa/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/fa/docs.tsv  
   creating: ./mewsli-9/output/dataset/ja/
  inflating: ./mewsli-9/output/dataset/ja/mentions.tsv  
  inflating: ./mewsli-9/output/dataset/ja/docs.tsv  
   creating: ./mewsli-9/output/dataset/sr/
  inflating: ./m

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1_W_B0kOG3bLnnXRluUa1sptUWRKcqsDy
To: /content/mel.zip
100%|██████████| 84.0k/84.0k [00:00<00:00, 67.4MB/s]
+ DATASET_DIR=./mewsli-9/output/dataset
+ mkdir -p ./mewsli-9/output/dataset
+ wget https://storage.googleapis.com/gresearch/mewsli/mewsli-9.zip
--2022-05-29 22:33:46--  https://storage.googleapis.com/gresearch/mewsli/mewsli-9.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.188.208, 142.251.33.208, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.188.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10155722 (9.7M) [application/zip]
Saving to: ‘mewsli-9.zip’

     0K .......... .......... .......... .......... .....

In [3]:
!mv mewsli-9/output/dataset .

In [4]:
import glob
import os
import pandas as pd
import json
import random
import numpy as np
import requests
from bs4 import BeautifulSoup

In [5]:
#filter those entities that are in knowledge base in at least 1 language (out of 9 taken)

val=[True,False]
def ch(x):
  if x == 'True':
    return True
  elif x == 'False':
    return False
  else:
    return x
data = pd.DataFrame({'qid' : [0],'qid_in_refs': [True]})
for nm in glob.glob('dataset/*/mentions.tsv')[:3]:
    men = pd.read_csv(nm,sep='\t', on_bad_lines='skip').dropna()
    men['qid_in_refs'] = men['qid_in_refs'].apply(lambda x: ch(x))
    men = men.query('qid_in_refs == True').drop_duplicates()
    data = data.append(men[['qid','qid_in_refs']], ignore_index=True)
  
data = data.iloc[1:]
good_qid = data.qid.values
good_qid = set(good_qid)
qid_to_ind=dict(zip(good_qid,np.arange(len(good_qid)).tolist()))

In [6]:
def get_context(link_):
  response = requests.get(link_)

  if response is not None:
      html = BeautifulSoup(response.text, 'html.parser')
      title = html.select("#firstHeading")[0].text
      paragraphs = html.select("p")

      intro = ''.join([ para.text for para in paragraphs[0:5]])
      intro = intro.replace('\n',' ')
  else:
      intro = None
  return intro

In [ ]:
# this cell creates 3 json files mentions_train.json mentions_test.json mentions_val.json

prev_index=0
full_dict={}
lang_folders = glob.glob(f'dataset/*/mentions.tsv')
print(lang_folders)
for nm in lang_folders:
    men = pd.read_csv(nm,sep='\t', on_bad_lines='skip').query('qid in @good_qid')
    men.reset_index(drop=True, inplace=True)
    lang=nm[nm.find(os.path.basename(nm))-1-2:nm.find(os.path.basename(nm))-1]
    print(lang)
    dir_path=os.path.join('dataset',lang)
    dict_mens_one_lang=dict()
    for index, row in men.iterrows():
        one_mention=dict()
        path=os.path.join(dir_path,'text',row.docid)
        with open(path, 'r') as file:
          lines = file.readlines()
          lines = [line.rstrip() for line in lines]

        title = lines[0].replace("'","").replace('"','')
        text=''.join(lines[1:])
        start = text.find(row.mention)            
        one_mention["start_index"] = start
        one_mention["end_index"] = start+row.length
        one_mention["mention_its"] = row.mention
        one_mention["document_id"] = row.docid
        one_mention["mention_id"] = row.qid
        one_mention["source_document"] = {
            "title": title,
            "text": text
          }
        one_mention["entity_context"] = get_context(row.url)
        true_index=index+prev_index
        dict_mens_one_lang[true_index] = one_mention
    full_dict.update(dict_mens_one_lang)
    print(len(dict_mens_one_lang))
    prev_index+=(index+1)

In [ ]:
test_size=0.15
val_size=0.15
random.seed(42)


all_ids = np.arange(len(full_dict))

test_indexes = random.sample(list(all_ids), int(len(all_ids)*test_size))

train_val_indexes = set(all_ids)-set(test_indexes)

val_indexes = random.sample(list(train_val_indexes), int(len(train_val_indexes)*val_size))

train_indexes = set(train_val_indexes)-set(val_indexes)

assert len(full_dict) == len(test_indexes)+len(val_indexes)+len(train_indexes)

In [ ]:
#test-------------------------------
values = [full_dict[key] for key in test_indexes]
keys=np.arange(len(test_indexes)).tolist()
test_dict = dict(zip(keys, values))

json_data = json.dumps(test_dict, indent=2, ensure_ascii=False)
with open('mentions_test.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

#val--------------------------------
values = [full_dict[key] for key in val_indexes]
keys=np.arange(len(val_indexes)).tolist()
val_dict = dict(zip(keys, values))

json_data = json.dumps(val_dict, indent=2, ensure_ascii=False)
with open('mentions_val.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

#train----------------------------
values = [full_dict[key] for key in train_indexes]
keys=np.arange(len(train_indexes)).tolist()
train_dict = dict(zip(keys, values))

json_data = json.dumps(train_dict, indent=2, ensure_ascii=False)
with open('mentions_train.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)